In [25]:
#Part - 3

#multisearch agent rag LLM application

#We will learn about Tools, Toolkit, Agents

!pip install arxiv
!pip install wikipedia
from langchain.tools import google_search

#output = google_search("Tools for multisearch agent rag LLM application")
#print(output)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [26]:


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

apiwraper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200, lang='en')

#print(apiwraper.query_and_summarize("Deep learning"))
tool_wikipedia = WikipediaQueryRun(api_wrapper=apiwraper)
tool_wikipedia.name

'wikipedia'

In [27]:
import os
from dotenv import load_dotenv

load_dotenv('../.env.py')
print(os.getenv('OPENAI_API_KEY'))
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings

loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever1 = vectordb.as_retriever()

retriever1


sk-proj-sDfBYiD0PHkbELY1n7OTT3BlbkFJjZdHRcNQXF3sz6JF8Qld


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13c4bea30>)

In [28]:
from langchain.tools.retriever import create_retriever_tool

documentprompt = ""
retriever_tool = create_retriever_tool(retriever1, "langsmith_search", "Search about langsmith as a framework, for any questions about langsmith capabilities or how to use langsmith")
retriever_tool.name


'langsmith_search'

In [29]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxivwrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool_arxiv = ArxivQueryRun(api_wrapper=arxivwrapper)
tool_arxiv.name

'arxiv'

In [30]:
tools = [retriever_tool, tool_arxiv, tool_wikipedia]
tools


[Tool(name='langsmith_search', description='Search about langsmith as a framework, for any questions about langsmith capabilities or how to use langsmith', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x1081571f0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13c4bea30>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1081573a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13c4bea30>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_ex

In [31]:
%pip install langchain_openai

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.6)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [32]:
#%pip install langchainhub
#Function Agents from hub for prompts

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
#Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [34]:
#Agent Executor

from langchain.agents import AgentExecutor
agentexecutor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agentexecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [35]:
#invoke Agent executor
agentexecutor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain

evaluation in the how-to guides.Was this page helpful?NextTutorials1. Install LangSmith2. Create an API key3. Set up your environment4. Log your first trace5. Run your first evaluationCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.

works on its own!1. 

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith works independently and does not require the use of LangChain. \n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n\n3. Set up your environment by exporting the necessary variables:\n   ```\n   export LANGCHAIN_TRACING_V2=true\n   export LANGCHAIN_API_KEY=<your-api-key>\n   export OPENAI_API_KEY=<your-openai-api-key>  # Example using the OpenAI API key\n   ```\n\n4. Log your first trace using the provided methods.\n\nLangSmith provides detailed documentation, tutorials, how-to guides, an

In [36]:
#invoke Agent executor
agentexecutor.invoke({"input":"Tell me about deep learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `deep learning`


Page: Deep learning
Summary: Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers inDeep learning is a subset of machine learning methods that are based on neural networks with representation learning. The term "deep" in deep learning refers to the use of multiple layers in neural networks. If you would like more detailed information or have specific questions about deep learning, feel free to ask!

> Finished chain.


{'input': 'Tell me about deep learning',
 'output': 'Deep learning is a subset of machine learning methods that are based on neural networks with representation learning. The term "deep" in deep learning refers to the use of multiple layers in neural networks. If you would like more detailed information or have specific questions about deep learning, feel free to ask!'}

In [37]:
#invoke Agent executor
agentexecutor.invoke({"input":"Modeling pedestrian fundamental diagram based on Directional Statistics"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Modeling pedestrian fundamental diagram based on Directional Statistics'}`


Published: 2024-05-02
Title: Modeling pedestrian fundamental diagram based on Directional Statistics
Authors: Keiichiro Fujiya, Kota Nagasaki, Toru Seo
Summary: Understanding pedestrian dynamics is crI found a paper titled "Modeling pedestrian fundamental diagram based on Directional Statistics" by Keiichiro Fujiya, Kota Nagasaki, and Toru Seo. The paper discusses understanding pedestrian dynamics. If you need more information or details from the paper, feel free to ask!

> Finished chain.


{'input': 'Modeling pedestrian fundamental diagram based on Directional Statistics',
 'output': 'I found a paper titled "Modeling pedestrian fundamental diagram based on Directional Statistics" by Keiichiro Fujiya, Kota Nagasaki, and Toru Seo. The paper discusses understanding pedestrian dynamics. If you need more information or details from the paper, feel free to ask!'}